In [ ]:
import pandas as pd
import os
import glob
import zipfile
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ydata_profiling import ProfileReport
from openpyxl import Workbook
from openpyxl.styles import Border, Side, Alignment, PatternFill, Font
from openpyxl.worksheet.page import PageMargins
from openpyxl.utils import get_column_letter
from google.colab import drive
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설치 및 설정
!apt-get install -y fonts-nanum
plt.rcParams['font.family'] = 'NanumGothic'

# 구글 드라이브를 마운트합니다.
drive.mount('/content/drive', force_remount=True)

# 압축 파일 경로 및 압축 해제 경로 설정
zip_file_path = '/content/drive/My Drive/회사업무/인허가자료_08월/LOCALDATA_NOWMON_CSV 3.zip'  # 실제 파일 경로로 변경
extract_folder = '/content/extracted_data'

# 폴더가 없으면 생성
os.makedirs(extract_folder, exist_ok=True)

# 압축 해제
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
except FileNotFoundError:
    print(f"Error: Specified zip file '{zip_file_path}' not found. Please check the path.")
    # Handle the error appropriately, e.g., exit the script or prompt for a different file path.

# 압축 해제된 폴더에서 모든 CSV 파일 목록 가져오기
all_files = glob.glob(os.path.join(extract_folder, "**/*.csv"), recursive=True)

if len(all_files) == 0:
    print("Warning: No CSV files found in the specified directory. Proceeding with an empty DataFrame.")
    dfs = [] # 빈 리스트로 초기화
else:
    # 각 파일을 읽어서 하나의 DataFrame으로 병합
    dfs = []
    for file in all_files:
        try:
            df = pd.read_csv(file, encoding='cp949', on_bad_lines='skip', dtype=str, low_memory=False)
            # 주소라는 키워드를 포함한 열 확인
            address_columns = [col for col in df.columns if '주소' in col]
            if not address_columns:
                print(f"No address column found in file: {file}")
                continue
            # '서울', '경기', '강원'이 포함된 행 필터링
            df_filtered = df[df[address_columns[0]].str.contains('서울|경기|강원', na=False)]
            dfs.append(df_filtered)
            print(f"Successfully read and filtered file: {file}")
        except Exception as e:
            print(f"Error reading file {file}: {e}")

if len(dfs) == 0:
    # 이전 단계에서 파일이 없을 경우 빈 DataFrame 생성
    concatenated_df = pd.DataFrame()
else:
    concatenated_df = pd.concat(dfs, ignore_index=True)

    # 중복된 행 제거: 사업장명, 소재지전체주소, 영업상태명이 동일한 경우 하나만 남기고 제거
    concatenated_df.drop_duplicates(subset=['사업장명', '소재지전체주소', '영업상태명'], inplace=True)

    # '인허가일자' 기준으로 내림차순 정렬
    if '인허가일자' in concatenated_df.columns:
        concatenated_df['인허가일자'] = pd.to_datetime(concatenated_df['인허가일자'], format='%Y%m%d', errors='coerce')
        concatenated_df.sort_values(by='인허가일자', ascending=False, inplace=True)

# 필요한 열만 선택하여 필터링
selected_columns = ['소재지전체주소', '도로명전체주소', '도로명우편번호', '사업장명', '개방서비스명', '인허가일자', '인허가취소', '영업상태명', '폐업일자',
                    '휴업시작일', '휴업종료일', '재개업일자', '소재지전화', '최종수정시점', '업태구분명', '좌표정보(X)', '좌표정보(Y)', '총면적', '소재지면적']

# 존재하는 열만 선택
existing_columns = [col for col in selected_columns if col in concatenated_df.columns]
filtered_df = concatenated_df[existing_columns]

# CSV 파일로 저장
output_dir = '/content/drive/My Drive/회사업무/인허가자료_08월'
os.makedirs(output_dir, exist_ok=True)
output_csv_path = os.path.join(output_dir, '1.0801_09까지변동_병합(서울,경기,강원).csv')
filtered_df.to_csv(output_csv_path, index=False, encoding='cp949')

# 엑셀 파일 경로 설정
file1_path = os.path.join('/content/drive/My Drive/회사업무/행안부자료/0801', '1.영업구역별_주소현행화0725.xlsx')
file2_path = output_csv_path  # 이전 단계에서 생성한 파일 사용

# 파일 경로 확인
if not os.path.exists(file1_path):
    raise FileNotFoundError(f"{file1_path} 파일이 존재하지 않습니다. 경로를 확인하세요.")
if not os.path.exists(file2_path):
    raise FileNotFoundError(f"{file2_path} 파일이 존재하지 않습니다. 경로를 확인하세요.")

# 엑셀 파일 읽기
df1 = pd.read_excel(file1_path)
df2 = pd.read_csv(file2_path, encoding='cp949')

# 주소 정규화 함수
def normalize_address(address):
    if pd.isna(address):
        return address
    address = address.strip()
    address = address.replace('강원특별자치도', '강원도')
    address = address.replace('서울특별시', '서울시')
    if '*' in address:
        return None
    return address

# df1의 주소 정규화
df1['full_address'] = df1[['주소시', '주소군구', '주소동']].astype(str).agg(' '.join, axis=1).apply(normalize_address)
df1 = df1.dropna(subset=['full_address'])

# df2의 주소 정규화
df2['소재지전체주소'] = df2['소재지전체주소'].astype(str).apply(normalize_address)
df2['도로명전체주소'] = df2['도로명전체주소'].astype(str).apply(normalize_address)
df2 = df2.dropna(subset=['소재지전체주소', '도로명전체주소'])

# TF-IDF 벡터화
vectorizer = TfidfVectorizer().fit(df1['full_address'])
tfidf_matrix = vectorizer.transform(df1['full_address'])

# 유사한 주소 매핑
def get_best_match(address, tfidf_matrix, vectorizer, choices, threshold=0.7):
    if pd.isna(address):
        return None
    query_vec = vectorizer.transform([address])
    cosine_similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    best_match_index = cosine_similarities.argmax()
    best_match_score = cosine_similarities[best_match_index]
    if best_match_score >= threshold:
        return choices[best_match_index]
    return None

choices = df1['full_address'].tolist()
df2['matched_address_소재지'] = df2['소재지전체주소'].apply(lambda x: get_best_match(x, tfidf_matrix, vectorizer, choices))
df2['matched_address_도로명'] = df2['도로명전체주소'].apply(lambda x: get_best_match(x, tfidf_matrix, vectorizer, choices))

df2['matched_address'] = df2.apply(lambda x: x['matched_address_소재지'] if pd.notna(x['matched_address_소재지']) else x['matched_address_도로명'], axis=1)

# 매핑되지 않은 항목 확인
unmatched = df2[df2['matched_address'].isna()]
print("매핑되지 않은 항목 수:", len(unmatched))

# 평수 계산을 위한 열 존재 확인 및 병합
merge_columns = ['full_address', '관리지사', 'SP담당']
if '총면적' in df1.columns:
    merge_columns.append('총면적')
if '소재지면적' in df1.columns:
    merge_columns.append('소재지면적')

df_merged = df2.merge(df1[merge_columns], left_on='matched_address', right_on='full_address', how='left', suffixes=('', '_df1'))

# 매핑 결과 확인
print("매핑 후 관리지사 및 SP담당이 비어있는 행 수:", df_merged[['관리지사', 'SP담당']].isna().sum())

# 평수 계산 (1평 = 3.305785 m^2)
def calculate_area(row):
    if '소재지면적' in row and pd.notna(row['소재지면적']):
        return round(row['소재지면적'] / 3.305785)
    elif '총면적' in row and pd.notna(row['총면적']):
        return round(row['총면적'] / 3.305785)
    else:
        return None

df_merged['평수'] = df_merged.apply(calculate_area, axis=1)

# 불필요한 열 제외하기 전 필요한 열만 선택
columns_to_keep = ['관리지사', 'SP담당', '사업장명', '개방서비스명', '업태구분명', '평수', '소재지전체주소', '도로명전체주소', '소재지전화', '폐업일자', '재개업일자', '영업상태명']
df_filtered = df_merged[columns_to_keep]

# 중복된 항목 제거
df_filtered.drop_duplicates(inplace=True)

# 평수 내림차순 정렬
df_filtered_sorted = df_filtered.sort_values(by='평수', ascending=False)

# NaN 값 처리
df_filtered_sorted['관리지사'] = df_filtered_sorted['관리지사'].fillna('Unknown')
df_filtered_sorted['SP담당'] = df_filtered_sorted['SP담당'].fillna('Unknown')

# SP담당자별 데이터 분리 및 파일 저장
def save_to_excel(df, file_path):
    with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
        # 전체 시트 저장
        df.to_excel(writer, index=False, sheet_name='전체')

        # 영업/정상 시트 저장
        df_active = df[df['영업상태명'].isin(['영업/정상'])]
        df_active.to_excel(writer, index=False, sheet_name='영업_정상')

        # 폐업 시트 저장
        df_closed = df[df['영업상태명'] == '폐업']
        df_closed.to_excel(writer, index=False, sheet_name='폐업')

        workbook = writer.book
        thin_border = Border(left=Side(style='thin', color='D3D3D3'),
                             right=Side(style='thin', color='D3D3D3'),
                             top=Side(style='thin', color='D3D3D3'),
                             bottom=Side(style='thin', color='D3D3D3'))
        header_fill = PatternFill(start_color="000080", end_color="000080", fill_type="solid")
        header_font = Font(color="FFFFFF", bold=True)

        # 스타일 설정 함수
        def set_style(worksheet):
            for cell in worksheet[1]:
                cell.fill = header_fill
                cell.font = header_font
                cell.border = thin_border
                cell.alignment = Alignment(horizontal='center', vertical='center')
            for row in worksheet.iter_rows(min_row=2):
                for cell in row:
                    cell.border = thin_border
                    cell.alignment = Alignment(horizontal='center', vertical='center')
            for col in worksheet.columns:
                max_length = 0
                column = col[0].column_letter
                for cell in col:
                    try:
                        if len(str(cell.value)) > max_length:
                            max_length = len(cell.value)
                    except:
                        pass
                adjusted_width = (max_length + 2)
                worksheet.column_dimensions[column].width = adjusted_width

        set_style(writer.sheets['전체'])
        set_style(writer.sheets['영업_정상'])
        set_style(writer.sheets['폐업'])

output_dir = '/content/drive/My Drive/회사업무/인허가자료_08월'
for manager in df_filtered_sorted['관리지사'].unique():
    manager_dir = os.path.join(output_dir, manager)
    os.makedirs(manager_dir, exist_ok=True)
    df_manager = df_filtered_sorted[df_filtered_sorted['관리지사'] == manager]

    for sp in df_manager['SP담당'].unique():
        df_sp = df_manager[df_manager['SP담당'] == sp]
        sp_file_path = os.path.join(manager_dir, f'{manager}_{sp}_0809까지.xlsx')
        save_to_excel(df_sp, sp_file_path)

# 전체 데이터를 저장
total_file_path = os.path.join(output_dir, '0801_09까지 전체_최종결과물.xlsx')
save_to_excel(df_filtered_sorted, total_file_path)

# 전체 데이터를 바탕으로 시각화
status_counts = df_filtered_sorted.groupby(['관리지사', '영업상태명']).size().unstack(fill_value=0)
status_counts = status_counts.loc[status_counts.sum(axis=1).sort_values(ascending=False).index]

# 시각화
fig, ax = plt.subplots(figsize=(14, 10))
status_counts.plot(kind='bar', stacked=True, ax=ax, color=['skyblue', 'salmon'])
ax.set_title('관리지사별 영업상태명 집계현황', fontsize=16)
ax.set_xlabel('관리지사', fontsize=12)
ax.set_ylabel('건수', fontsize=12)
ax.legend(title='영업상태명', fontsize=10, title_fontsize='13')
plt.xticks(rotation=90)
plt.tight_layout()

# 그래프를 이미지로 저장
fig_path = os.path.join(output_dir, '전체_영업상태명_집계현황.png')
fig.savefig(fig_path, bbox_inches='tight')

# 그래프를 출력
plt.show()

# EDA 보고서 생성
profile = ProfileReport(df_filtered_sorted, title="EDA Report", explorative=True)
eda_report_path = os.path.join(output_dir, 'EDA_Report.html')
profile.to_file(eda_report_path)

print(f"전체 매핑된 파일이 생성되었습니다: {total_file_path}")
print(f"EDA 보고서가 생성되었습니다: {eda_report_path}")
print(f"전체 영업상태명 집계현황 그래프가 생성되었습니다: {fig_path}")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


ValueError: mount failed

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

ValueError: flush_and_unmount failed

In [ ]:
!pip install ydata-profiling

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

ValueError: flush_and_unmount failed

In [ ]:
# prompt: ValueError                                Traceback (most recent call last)
# <ipython-input-5-40b4fb998ea5> in <cell line: 41>()
#      40
#      41 if len(all_files) == 0:
# ---> 42     raise ValueError("No CSV files found in the specified directory.")
#      43
#      44 # 각 파일을 읽어서 하나의 DataFrame으로 병합
# ValueError: No CSV files found in the specified directory.

import pandas as pd
import os
import glob
import zipfile
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ydata_profiling import ProfileReport
from openpyxl import Workbook
from openpyxl.styles import Border, Side, Alignment, PatternFill, Font
from openpyxl.worksheet.page import PageMargins
from openpyxl.utils import get_column_letter
from google.colab import drive
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설치 및 설정
!apt-get install -y fonts-nanum
plt.rcParams['font.family'] = 'NanumGothic'

# 구글 드라이브를 마운트합니다.
drive.mount('/content/drive', force_remount=True)

# 압축 파일 경로 및 압축 해제 경로 설정
zip_file_path = '/content/drive/My Drive/회사업무/인허가자료_08월/LOCALDATA_NOWMON_CSV 3.zip'  # 실제 파일 경로로 변경
extract_folder = '/content/extracted_data'

# 폴더가 없으면 생성
os.makedirs(extract_folder, exist_ok=True)

# 압축 해제
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
except FileNotFoundError:
    print(f"Error: Specified zip file '{zip_file_path}' not found. Please check the path.")
    # Handle the error appropriately, e.g., exit the script or prompt for a different file path.

# 압축 해제된 폴더에서 모든 CSV 파일 목록 가져오기
all_files = glob.glob(os.path.join(extract_folder, "**/*.csv"), recursive=True)

if len(all_files) == 0:
    print("Warning: No CSV files found in the specified directory. Proceeding with an empty DataFrame.")
    dfs = [] # 빈 리스트로 초기화
else:
    # 각 파일을 읽어서 하나의 DataFrame으로 병합
    dfs = []
    for file in all_files:
        try:
            df = pd.read_csv(file, encoding='cp949', on_bad_lines='skip', dtype=str, low_memory=False)
            # 주소라는 키워드를 포함한 열 확인
            address_columns = [col for col in df.columns if '주소' in col]
            if not address_columns:
                print(f"No address column found in file: {file}")
                continue
            # '서울', '경기', '강원'이 포함된 행 필터링
            df_filtered = df[df[address_columns[0]].str.contains('서울|경기|강원', na=False)]
            dfs.append(df_filtered)
            print(f"Successfully read and filtered file: {file}")
        except Exception as e:
            print(f"Error reading file {file}: {e}")

if len(dfs) == 0:
    # 이전 단계에서 파일이 없을 경우 빈 DataFrame 생성
    concatenated_df = pd.DataFrame()
else:
    concatenated_df = pd.concat(dfs, ignore_index=True)

    # 중복된 행 제거
    concatenated_df.drop_duplicates(inplace=True)

    # '인허가일자' 기준으로 내림차순 정렬
    if '인허가일자' in concatenated_df.columns:
        concatenated_df['인허가일자'] = pd.to_datetime(concatenated_df['인허가일자'], format='%Y%m%d', errors='coerce')
        concatenated_df.sort_values(by='인허가일자', ascending=False, inplace=True)

# ... (나머지 코드는 동일하게 유지)


In [ ]:
# prompt: ModuleNotFoundError                       Traceback (most recent call last)
# <ipython-input-1-70c0514d974b> in <cell line: 7>()
#       5 from sklearn.feature_extraction.text import TfidfVectorizer
#       6 from sklearn.metrics.pairwise import cosine_similarity
# ----> 7 from ydata_profiling import ProfileReport
#       8 from openpyxl import Workbook
#       9 from openpyxl.styles import Border, Side, Alig

!pip install ydata-profiling

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import matplotlib.font_manager as fm

# 한글 폰트 설치 및 설정
!apt-get install -y fonts-nanum
!fc-cache -fv

# 한글 폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=12)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 폰트 설정

# 구글 드라이브를 마운트합니다.
drive.mount('/content/drive', force_remount=True)

# 정확한 파일 경로 입력
file_path = '/content/drive/My Drive/회사업무/인허가자료_08월/0801_09까지 전체_최종결과물.xlsx'

if not os.path.exists(file_path):
    raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다. 경로를 확인하세요.")

# 데이터 프레임 읽기
df_filtered_sorted = pd.read_excel(file_path)

# '중앙' 지사 및 'Unknown' 제외
df_filtered_sorted = df_filtered_sorted[(df_filtered_sorted['관리지사'] != 'Unknown') &
                                        (df_filtered_sorted['SP담당'] != 'Unknown')]

# '영업/정상'과 '폐업' 상태만 필터링
df_filtered_sorted = df_filtered_sorted[df_filtered_sorted['영업상태명'].isin(['영업/정상', '폐업'])]

# 매핑된 데이터만 필터링
df_filtered_sorted = df_filtered_sorted.dropna(subset=['관리지사', 'SP담당'])

# 관리지사별로 집계
grouped_df = df_filtered_sorted.groupby(['관리지사', '영업상태명']).size().unstack(fill_value=0)

# 원하는 순서로 관리지사 정렬
desired_order = ['중앙지사', '강북지사', '서대문지사', '고양지사', '의정부지사', '남양주지사', '원주지사', '강릉지사']
grouped_df = grouped_df.reindex(desired_order)

# 관리지사별 현황표 생성
print("관리지사별 영업상태 현황표:")
print(grouped_df)

# 시각화 (막대 그래프)
fig, ax = plt.subplots(figsize=(20, 14))  # 그래프 크기를 크게 조정
colors = ['#4caf50', '#f44336']  # 영업/정상 = 녹색, 폐업 = 빨간색

# 막대 그래프 생성
bars = grouped_df.plot(kind='bar', stacked=True, ax=ax, color=colors, width=0.8)

# 그래프 제목 및 레이블 설정
ax.set_title('관리지사별 영업상태명 집계현황', fontsize=22, fontproperties=fontprop)
ax.set_xlabel('관리지사', fontsize=16, fontproperties=fontprop)
ax.set_ylabel('건수', fontsize=16, fontproperties=fontprop)
plt.xticks(fontsize=14, fontproperties=fontprop, rotation=0, ha='center')  # 텍스트 수평 정렬
plt.yticks(fontsize=14, fontproperties=fontprop)

# 막대에 숫자 표시
for container in bars.containers:
    bars.bar_label(container, label_type='center', fontsize=12, fontproperties=fontprop, color='white' if container.get_label() == '영업/정상' else 'black')

# 범례 설정
handles, labels = ax.get_legend_handles_labels()
legend = ax.legend(handles, labels, title='영업상태명', fontsize=12, title_fontsize='14', prop=fontprop, loc='upper right', frameon=True)
plt.setp(legend.get_title(), fontproperties=fontprop)

# 범례 배경과 테두리 설정
legend.get_frame().set_facecolor('white')  # 배경색
legend.get_frame().set_edgecolor('black')  # 테두리색

# 그래프에 그리드 추가
ax.grid(True, axis='y', linestyle='--', alpha=0.7)

# 그래프를 이미지로 저장
output_dir = '/content/drive/My Drive/회사업무/인허가자료_08월'
fig_path = os.path.join(output_dir, '관리지사별_영업상태명_집계현황_정렬.png')
fig.savefig(fig_path, bbox_inches='tight')

# 그래프를 출력
plt.show()

print(f"관리지사별 영업상태명 집계현황 그래프가 생성되었습니다: {fig_path}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import matplotlib.font_manager as fm

# 한글 폰트 설치 및 설정
!apt-get install -y fonts-nanum
!fc-cache -fv

# 한글 폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=12)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 폰트 설정

# 구글 드라이브를 마운트합니다.
drive.mount('/content/drive', force_remount=True)

# 정확한 파일 경로 입력
file_path = '/content/drive/My Drive/회사업무/인허가자료_08월/0801_09까지 전체_최종결과물.xlsx'

if not os.path.exists(file_path):
    raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다. 경로를 확인하세요.")

# 데이터 프레임 읽기
df_filtered_sorted = pd.read_excel(file_path)

# '중앙' 지사 및 'Unknown' 제외
df_filtered_sorted = df_filtered_sorted[(df_filtered_sorted['관리지사'] != '중앙') &
                                        (df_filtered_sorted['관리지사'] != 'Unknown') &
                                        (df_filtered_sorted['SP담당'] != 'Unknown')]

# 담당자 앞에 관리지사 이름 추가 (지사 제외)
df_filtered_sorted['SP담당자'] = df_filtered_sorted['관리지사'].str.replace('지사', '') + '-' + df_filtered_sorted['SP담당']

# '영업/정상'과 '폐업' 상태만 필터링
df_filtered_sorted = df_filtered_sorted[df_filtered_sorted['영업상태명'].isin(['영업/정상', '폐업'])]

# 매핑된 데이터만 필터링
df_filtered_sorted = df_filtered_sorted.dropna(subset=['관리지사', 'SP담당'])

# 전체 데이터를 바탕으로 시각화
status_counts = df_filtered_sorted.groupby(['SP담당자', '영업상태명']).size().unstack(fill_value=0)
status_counts = status_counts.loc[status_counts.sum(axis=1).sort_values(ascending=True).index]

# 시각화 (가로 막대 그래프)
fig, ax = plt.subplots(figsize=(16, 12))
colors = ['#4caf50', '#f44336']  # 영업/정상 = 녹색, 폐업 = 빨간색

# 막대 그래프 생성
bars = status_counts.plot(kind='barh', stacked=True, ax=ax, color=colors, width=0.8)

# 그래프 제목 및 레이블 설정
ax.set_title('SP담당자별 영업상태명 집계현황', fontsize=18, fontproperties=fontprop)
ax.set_xlabel('건수', fontsize=14, fontproperties=fontprop)
ax.set_ylabel('SP담당자', fontsize=14, fontproperties=fontprop)
plt.xticks(fontsize=12, fontproperties=fontprop)
plt.yticks(fontsize=12, fontproperties=fontprop)

# 막대에 숫자 표시
for container in bars.containers:
    bars.bar_label(container, label_type='center', fontsize=10, fontproperties=fontprop, color='white' if container.get_label() == '영업/정상' else 'black')

# 범례 설정
handles, labels = ax.get_legend_handles_labels()
legend = ax.legend(handles, labels, title='영업상태명', fontsize=12, title_fontsize='14', prop=fontprop, loc='lower right')
plt.setp(legend.get_title(), fontproperties=fontprop)

# 그래프에 그리드 추가
ax.grid(True, axis='x', linestyle='--', alpha=0.7)

# 그래프를 이미지로 저장
output_dir = '/content/drive/My Drive/회사업무/인허가자료_08월'
fig_path = os.path.join(output_dir, '전체_영업상태명_집계현황_전문가.png')
fig.savefig(fig_path, bbox_inches='tight')

# 그래프를 출력
plt.show()

print(f"전체 영업상태명 집계현황 전문가 그래프가 생성되었습니다: {fig_path}")

In [ ]:
!pip install python-pptx

In [ ]:
from pptx import Presentation
   from pptx.util import Inches

   # 새로운 프레젠테이션 생성
   prs = Presentation()

   # 첫 번째 슬라이드: 제목 슬라이드
   slide_layout = prs.slide_layouts[0]
   slide = prs.slides.add_slide(slide_layout)
   title = slide.shapes.title
   subtitle = slide.placeholders[1]

   title.text = "프로젝트 개요"
   subtitle.text = "데이터 분석 결과 보고서"

   # 두 번째 슬라이드: 내용 슬라이드
   slide_layout = prs.slide_layouts[1]
   slide = prs.slides.add_slide(slide_layout)
   title = slide.shapes.title
   content = slide.placeholders[1]

   title.text = "프로젝트 목표"
   content.text = "관리지사별 영업상태 분석 및 경영진 보고"

   # 세 번째 슬라이드: 그래프 슬라이드 (그래프 이미지 추가)
   slide_layout = prs.slide_layouts[5]  # 타이틀 + 내용만 있는 레이아웃
   slide = prs.slides.add_slide(slide_layout)
   title = slide.shapes.title
   title.text = "관리지사별 영업상태 분석 결과"

   # 이미지(그래프) 추가 (예시: 생성된 그래프 이미지 경로)
   img_path = '/content/drive/My Drive/회사업무/인허가자료_08월/관리지사별_영업상태명_집계현황_정렬.png'
   slide.shapes.add_picture(img_path, Inches(1), Inches(2), width=Inches(8), height=Inches(4.5))

   # 네 번째 슬라이드: 결론 슬라이드
   slide_layout = prs.slide_layouts[1]
   slide = prs.slides.add_slide(slide_layout)
   title = slide.shapes.title
   content = slide.placeholders[1]

   title.text = "결론 및 제안"
   content.text = "데이터 분석 결과를 바탕으로 경영진에게 제안"

   # PPT 파일 저장
   ppt_file_path = '/content/drive/My Drive/회사업무/인허가자료_08월/영업상태_분석_결과.pptx'
   prs.save(ppt_file_path)

   print(f"PPT 파일이 생성되었습니다: {ppt_file_path}")


In [ ]:
# 필요한 라이브러리 설치 및 로드
!pip install python-pptx

from pptx import Presentation
from pptx.util import Inches

# 새로운 프레젠테이션 생성
prs = Presentation()

# 첫 번째 슬라이드: 제목 슬라이드
slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(slide_layout)
title = slide.shapes.title
subtitle = slide.placeholders[1]

title.text = "프로젝트 개요"
subtitle.text = "데이터 분석 결과 보고서"

# 두 번째 슬라이드: 내용 슬라이드
slide_layout = prs.slide_layouts[1]
slide = prs.slides.add_slide(slide_layout)
title = slide.shapes.title
content = slide.placeholders[1]

title.text = "프로젝트 목표"
content.text = "관리지사별 영업상태 분석 및 경영진 보고"

# 세 번째 슬라이드: 그래프 슬라이드 (그래프 이미지 추가)
slide_layout = prs.slide_layouts[5]  # 타이틀 + 내용만 있는 레이아웃
slide = prs.slides.add_slide(slide_layout)
title = slide.shapes.title
title.text = "관리지사별 영업상태 분석 결과"

# 이미지(그래프) 추가 (예시: 생성된 그래프 이미지 경로)
img_path = '/content/drive/My Drive/회사업무/인허가자료_08월/관리지사별_영업상태명_집계현황_정렬.png'
slide.shapes.add_picture(img_path, Inches(1), Inches(2), width=Inches(8), height=Inches(4.5))

# 네 번째 슬라이드: 결론 슬라이드
slide_layout = prs.slide_layouts[1]
slide = prs.slides.add_slide(slide_layout)
title = slide.shapes.title
content = slide.placeholders[1]

title.text = "결론 및 제안"
content.text = "데이터 분석 결과를 바탕으로 경영진에게 제안"

# PPT 파일 저장
ppt_file_path = '/content/drive/My Drive/회사업무/인허가자료_08월/영업상태_분석_결과.pptx'
prs.save(ppt_file_path)

print(f"PPT 파일이 생성되었습니다: {ppt_file_path}")

In [ ]:
구글 코랩에서 직접 PPT를 생성할 수 있는 방법은 Python을 활용하여 `python-pptx` 라이브러리를 사용하는 것입니다. 이 라이브러리를 사용하면 슬라이드를 만들고, 텍스트, 이미지, 그래프 등을 추가할 수 있습니다.

다음은 구글 코랩에서 PPT 파일을 생성하는 예제 코드입니다.

1. **구글 코랩에서 `python-pptx` 설치:**

   ```python
   !pip install python-pptx
   ```

2. **PPT 파일 생성 및 기본 슬라이드 추가:**

   ```python
   from pptx import Presentation
   from pptx.util import Inches

   # 새로운 프레젠테이션 생성
   prs = Presentation()

   # 첫 번째 슬라이드: 제목 슬라이드
   slide_layout = prs.slide_layouts[0]
   slide = prs.slides.add_slide(slide_layout)
   title = slide.shapes.title
   subtitle = slide.placeholders[1]

   title.text = "프로젝트 개요"
   subtitle.text = "데이터 분석 결과 보고서"

   # 두 번째 슬라이드: 내용 슬라이드
   slide_layout = prs.slide_layouts[1]
   slide = prs.slides.add_slide(slide_layout)
   title = slide.shapes.title
   content = slide.placeholders[1]

   title.text = "프로젝트 목표"
   content.text = "관리지사별 영업상태 분석 및 경영진 보고"

   # 세 번째 슬라이드: 그래프 슬라이드 (그래프 이미지 추가)
   slide_layout = prs.slide_layouts[5]  # 타이틀 + 내용만 있는 레이아웃
   slide = prs.slides.add_slide(slide_layout)
   title = slide.shapes.title
   title.text = "관리지사별 영업상태 분석 결과"

   # 이미지(그래프) 추가 (예시: 생성된 그래프 이미지 경로)
   img_path = '/content/drive/My Drive/회사업무/인허가자료_08월/관리지사별_영업상태명_집계현황_정렬.png'
   slide.shapes.add_picture(img_path, Inches(1), Inches(2), width=Inches(8), height=Inches(4.5))

   # 네 번째 슬라이드: 결론 슬라이드
   slide_layout = prs.slide_layouts[1]
   slide = prs.slides.add_slide(slide_layout)
   title = slide.shapes.title
   content = slide.placeholders[1]

   title.text = "결론 및 제안"
   content.text = "데이터 분석 결과를 바탕으로 경영진에게 제안"

   # PPT 파일 저장
   ppt_file_path = '/content/drive/My Drive/회사업무/인허가자료_08월/영업상태_분석_결과.pptx'
   prs.save(ppt_file_path)

   print(f"PPT 파일이 생성되었습니다: {ppt_file_path}")
   ```

3. **PPT 파일 생성 완료 후 다운로드:**

   위 코드를 실행하면 PPT 파일이 지정된 경로에 생성됩니다. 해당 파일을 구글 드라이브에서 다운로드하거나 직접 열어볼 수 있습니다.

### 추가 사항:
- **슬라이드 레이아웃

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import font_manager as fm

# 한글 폰트 설정 (구글 코랩 기본 폰트 중 하나 사용)
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_prop = fm.FontProperties(fname=font_path, size=12)

# 다이어그램 크기 설정
fig, ax = plt.subplots(figsize=(10, 8))

# 박스 스타일 설정
box_style = dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="#d9eaf7")

# 텍스트 위치와 내용 설정
texts = [
    ("1. 설치 및 환경 설정", 0.5, 0.9),
    ("- python-pptx 설치\n- 라이브러리 불러오기", 0.5, 0.8),
    ("2. PPT 객체 생성", 0.5, 0.7),
    ("- Presentation 객체 생성", 0.5, 0.6),
    ("3. 슬라이드 추가", 0.5, 0.5),
    ("- 슬라이드 레이아웃 선택\n- 제목, 텍스트, 이미지 추가", 0.5, 0.4),
    ("4. PPT 저장", 0.5, 0.3),
    ("- 경로 지정\n- 파일 저장", 0.5, 0.2),
    ("5. 결과 확인", 0.5, 0.1),
    ("- 파일 생성 확인", 0.5, 0.05)
]

# 박스와 화살표 그리기
for text, x, y in texts:
    ax.text(x, y, text, transform=ax.transAxes, fontsize=12, verticalalignment="center", horizontalalignment="center", bbox=box_style, fontproperties=font_prop)

# 화살표 그리기
arrow_style = dict(facecolor="black", shrink=0.05, width=0.01, headwidth=8)

for i in range(len(texts) - 2):
    ax.annotate("", xy=(0.5, texts[i+1][2] - 0.03), xytext=(0.5, texts[i][2] + 0.03),
                xycoords="axes fraction", textcoords="axes fraction", arrowprops=arrow_style)

# 화살표 마지막 부분
ax.annotate("", xy=(0.5, texts[-1][2] - 0.01), xytext=(0.5, texts[-2][2] + 0.03),
            xycoords="axes fraction", textcoords="axes fraction", arrowprops=arrow_style)

# 축과 테두리 없애기
ax.axis("off")

# 다이어그램 출력
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import font_manager as fm

# 기본 폰트 설정 (나눔고딕이 아니라도, 기본 폰트를 활용)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 폰트 설정

# 다이어그램 크기 설정
fig, ax = plt.subplots(figsize=(12, 10))

# 박스 스타일 설정
box_style = dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="#d9eaf7")

# 텍스트 위치와 내용 설정
texts = [
    ("1. 데이터 로드", 0.5, 0.9),
    ("- Excel 파일에서 데이터 읽기", 0.5, 0.85),
    ("2. 데이터 필터링", 0.5, 0.75),
    ("- '중앙' 지사 및 'Unknown' 값 제거\n- '영업/정상' 및 '폐업' 상태 필터링", 0.5, 0.65),
    ("3. 매핑된 데이터 필터링", 0.5, 0.55),
    ("- 관리지사 및 SP담당 컬럼에서 NaN 값 제거", 0.5, 0.5),
    ("4. 데이터 집계", 0.5, 0.4),
    ("- 관리지사별로 영업상태 집계", 0.5, 0.35),
    ("5. 데이터 정렬", 0.5, 0.25),
    ("- 원하는 순서로 관리지사 데이터 정렬", 0.5, 0.2),
    ("6. 결과 시각화", 0.5, 0.1),
    ("- 막대 그래프로 결과 시각화", 0.5, 0.05),
    ("7. PPT 생성", 0.5, -0.05),
    ("- 결과를 PPT 슬라이드에 추가하여 저장", 0.5, -0.1)
]

# 박스와 화살표 그리기
for text, x, y in texts:
    ax.text(x, y, text, transform=ax.transAxes, fontsize=12, verticalalignment="center", horizontalalignment="center", bbox=box_style)

# 화살표 그리기
arrow_style = dict(facecolor="black", shrink=0.05, width=0.01, headwidth=8)

for i in range(0, len(texts)-2, 2):
    ax.annotate("", xy=(0.5, texts[i+2][2] + 0.03), xytext=(0.5, texts[i][2] - 0.03),
                xycoords="axes fraction", textcoords="axes fraction", arrowprops=arrow_style)

# 축과 테두리 없애기
ax.axis("off")

# 다이어그램 출력
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import font_manager as fm

# 한글 폰트 설정 (구글 코랩 기본 폰트 사용)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 폰트 설정

# 다이어그램 크기 설정
fig, ax = plt.subplots(figsize=(12, 12))

# 박스 스타일 설정
box_style = dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="#d9eaf7")

# 텍스트 위치와 내용 설정
texts = [
    ("1. 데이터 로드", 0.5, 0.9),
    ("- Excel 파일 경로 지정\n- pandas로 데이터 읽기", 0.5, 0.85),
    ("2. 데이터 필터링", 0.5, 0.75),
    ("- '중앙' 지사 및 'Unknown' 값 제거\n- '영업/정상' 및 '폐업' 상태 필터링", 0.5, 0.7),
    ("3. 매핑된 데이터 필터링", 0.5, 0.6),
    ("- 관리지사 및 SP담당 컬럼에서 NaN 값 제거", 0.5, 0.55),
    ("4. 데이터 집계", 0.5, 0.45),
    ("- 관리지사별로 영업상태 집계", 0.5, 0.4),
    ("5. 데이터 정렬", 0.5, 0.3),
    ("- 원하는 순서로 관리지사 데이터 정렬", 0.5, 0.25),
    ("6. 결과 시각화", 0.5, 0.15),
    ("- 막대 그래프로 결과 시각화\n- 그래프에 레이블 추가", 0.5, 0.1),
    ("7. PPT 생성", 0.5, 0.0),
    ("- matplotlib으로 생성된 그래프 이미지 추가\n- python-pptx로 슬라이드 생성 및 저장", 0.5, -0.05)
]

# 박스와 화살표 그리기
for text, x, y in texts:
    ax.text(x, y, text, transform=ax.transAxes, fontsize=12, verticalalignment="center", horizontalalignment="center", bbox=box_style)

# 화살표 그리기
arrow_style = dict(facecolor="black", shrink=0.05, width=0.01, headwidth=8)

for i in range(0, len(texts)-2, 2):
    ax.annotate("", xy=(0.5, texts[i+2][2] + 0.03), xytext=(0.5, texts[i][2] - 0.03),
                xycoords="axes fraction", textcoords="axes fraction", arrowprops=arrow_style)

# 축과 테두리 없애기
ax.axis("off")

# 다이어그램 출력
plt.show()

In [ ]:
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>데이터 가공 및 분석 프로세스</title>
    <style>
        body {
            font-family: 'Nanum Gothic', sans-serif;
            background-color: #f4f4f4;
            margin: 0;
            padding: 20px;
        }
        .container {
            max-width: 800px;
            margin: auto;
            background: #ffffff;
            padding: 20px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            border-radius: 8px;
        }
        h1 {
            text-align: center;
            font-size: 24px;
            color: #333333;
        }
        .step {
            margin-bottom: 20px;
        }
        .step h2 {
            font-size: 20px;
            color: #2c3e50;
            margin-bottom: 10px;
        }
        .step p {
            font-size: 16px;
            color: #34495e;
            line-height: 1.6;
            margin: 0;
            padding: 0;
        }
        .arrow {
            text-align: center;
            font-size: 20px;
            margin: 20px 0;
        }
    </style>
</head>
<body>

<div class="container">
    <h1>데이터 가공 및 분석 프로세스</h1>

    <div class="step">
        <h2>1. 데이터 로드</h2>
        <p>- Excel 파일 경로 지정<br>- pandas로 데이터 읽기</p>
    </div>

    <div class="arrow">↓</div>

    <div class="step">
        <h2>2. 데이터 필터링</h2>
        <p>- '중앙' 지사 및 'Unknown' 값 제거<br>- '영업/정상' 및 '폐업' 상태 필터링</p>
    </div>

    <div class="arrow">↓</div>

    <div class="step">
        <h2>3. 매핑된 데이터 필터링</h2>
        <p>- 관리지사 및 SP담당 컬럼에서 NaN 값 제거</p>
    </div>

    <div class="arrow">↓</div>

    <div class="step">
        <h2>4. 데이터 집계</h2>
        <p>- 관리지사별로 영업상태 집계</p>
    </div>

    <div class="arrow">↓</div>

    <div class="step">
        <h2>5. 데이터 정렬</h2>
        <p>- 원하는 순서로 관리지사 데이터 정렬</p>
    </div>

    <div class="arrow">↓</div>

    <div class="step">
        <h2>6. 결과 시각화</h2>
        <p>- 막대 그래프로 결과 시각화<br>- 그래프에 레이블 추가</p>
    </div>

    <div class="arrow">↓</div>

    <div class="step">
        <h2>7. PPT 생성</h2>
        <p>- matplotlib으로 생성된 그래프 이미지 추가<br>- python-pptx로 슬라이드 생성 및 저장</p>
    </div>
</div>

</body>
</html>

In [ ]:
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>데이터 가공 및 분석 프로세스</title>
    <style>
        body {
            font-family: 'Nanum Gothic', sans-serif;
            background-color: #f4f4f4;
            margin: 0;
            padding: 20px;
        }
        .container {
            max-width: 800px;
            margin: auto;
            background: #ffffff;
            padding: 20px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            border-radius: 8px;
        }
        h1 {
            text-align: center;
            font-size: 24px;
            color: #333333;
        }
        .step {
            margin-bottom: 20px;
        }
        .step h2 {
            font-size: 20px;
            color: #2c3e50;
            margin-bottom: 10px;
        }
        .step p {
            font-size: 16px;
            color: #34495e;
            line-height: 1.6;
            margin: 0;
            padding: 0;
        }
        .arrow {
            text-align: center;
            font-size: 20px;
            margin: 20px 0;
        }
    </style>
</head>
<body>

<div class="container">
    <h1>데이터 가공 및 분석 프로세스</h1>

    <div class="step">
        <h2>1. 데이터 로드</h2>
        <p>- Excel 파일 경로 지정<br>- pandas로 데이터 읽기</p>
    </div>

    <div class="arrow">↓</div>

    <div class="step">
        <h2>2. 데이터 필터링</h2>
        <p>- '중앙' 지사 및 'Unknown' 값 제거<br>- '영업/정상' 및 '폐업' 상태 필터링</p>
    </div>

    <div class="arrow">↓</div>

    <div class="step">
        <h2>3. 매핑된 데이터 필터링</h2>
        <p>- 관리지사 및 SP담당 컬럼에서 NaN 값 제거</p>
    </div>

    <div class="arrow">↓</div>

    <div class="step">
        <h2>4. 데이터 집계</h2>
        <p>- 관리지사별로 영업상태 집계</p>
    </div>

    <div class="arrow">↓</div>

    <div class="step">
        <h2>5. 데이터 정렬</h2>
        <p>- 원하는 순서로 관리지사 데이터 정렬</p>
    </div>

    <div class="arrow">↓</div>

    <div class="step">
        <h2>6. 결과 시각화</h2>
        <p>- 막대 그래프로 결과 시각화<br>- 그래프에 레이블 추가</p>
    </div>

    <div class="arrow">↓</div>

    <div class="step">
        <h2>7. PPT 생성</h2>
        <p>- matplotlib으로 생성된 그래프 이미지 추가<br>- python-pptx로 슬라이드 생성 및 저장</p>
    </div>
</div>

</body>
</html>

In [ ]:
import pandas as pd
import os
import glob
import zipfile
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Border, Side, Alignment, PatternFill, Font
from google.colab import drive
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설치 및 설정
!apt-get install -y fonts-nanum
plt.rcParams['font.family'] = 'NanumGothic'

# 구글 드라이브를 마운트합니다.
drive.mount('/content/drive', force_remount=True)

# 압축 파일 경로 및 압축 해제 경로 설정
zip_file_path = '/content/drive/My Drive/회사업무/인허가자료_08월/LOCALDATA_NOWMON_CSV 3.zip'  # 실제 파일 경로로 변경
extract_folder = '/content/extracted_data'

# 폴더가 없으면 생성
os.makedirs(extract_folder, exist_ok=True)

# 압축 해제
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
except FileNotFoundError:
    print(f"Error: Specified zip file '{zip_file_path}' not found. Please check the path.")
    exit(1)

# 압축 해제된 폴더에서 모든 CSV 파일 목록 가져오기
all_files = glob.glob(os.path.join(extract_folder, "**/*.csv"), recursive=True)

if len(all_files) == 0:
    print("Warning: No CSV files found in the specified directory. Proceeding with an empty DataFrame.")
    dfs = []  # 빈 리스트로 초기화
else:
    # 각 파일을 읽어서 하나의 DataFrame으로 병합
    dfs = []
    for file in all_files:
        try:
            df = pd.read_csv(file, encoding='cp949', on_bad_lines='skip', dtype=str, low_memory=False)
            # 주소라는 키워드를 포함한 열 확인
            address_columns = [col for col in df.columns if '주소' in col]
            if not address_columns:
                print(f"No address column found in file: {file}")
                continue
            # '서울', '경기', '강원'이 포함된 행 필터링
            df_filtered = df[df[address_columns[0]].str.contains('서울|경기|강원', na=False)]
            dfs.append(df_filtered)
            print(f"Successfully read and filtered file: {file}")
        except Exception as e:
            print(f"Error reading file {file}: {e}")

if len(dfs) == 0:
    concatenated_df = pd.DataFrame()  # 이전 단계에서 파일이 없을 경우 빈 DataFrame 생성
else:
    concatenated_df = pd.concat(dfs, ignore_index=True)
    # 중복된 행 제거: 사업장명, 소재지전체주소, 영업상태명이 동일한 경우 하나만 남기고 제거
    concatenated_df.drop_duplicates(subset=['사업장명', '소재지전체주소', '영업상태명'], inplace=True)

    # '인허가일자' 기준으로 내림차순 정렬
    if '인허가일자' in concatenated_df.columns:
        concatenated_df['인허가일자'] = pd.to_datetime(concatenated_df['인허가일자'], format='%Y%m%d', errors='coerce')
        concatenated_df.sort_values(by='인허가일자', ascending=False, inplace=True)

# 필요한 열만 선택하여 필터링
selected_columns = ['소재지전체주소', '도로명전체주소', '도로명우편번호', '사업장명', '개방서비스명', '인허가일자', '인허가취소', '영업상태명', '폐업일자',
                    '휴업시작일', '휴업종료일', '재개업일자', '소재지전화', '최종수정시점', '업태구분명', '좌표정보(X)', '좌표정보(Y)', '총면적', '소재지면적']

# 존재하는 열만 선택
existing_columns = [col for col in selected_columns if col in concatenated_df.columns]
filtered_df = concatenated_df[existing_columns]

# CSV 파일로 저장
output_dir = '/content/drive/My Drive/회사업무/인허가자료_08월'
os.makedirs(output_dir, exist_ok=True)
output_csv_path = os.path.join(output_dir, '1.0801_09까지변동_병합(서울,경기,강원).csv')
filtered_df.to_csv(output_csv_path, index=False, encoding='cp949')

# 엑셀 파일 경로 설정
file1_path = os.path.join('/content/drive/My Drive/회사업무/행안부자료/0801', '1.영업구역별_주소현행화0725.xlsx')
file2_path = output_csv_path  # 이전 단계에서 생성한 파일 사용

# 파일 경로 확인
if not os.path.exists(file1_path):
    raise FileNotFoundError(f"{file1_path} 파일이 존재하지 않습니다. 경로를 확인하세요.")
if not os.path.exists(file2_path):
    raise FileNotFoundError(f"{file2_path} 파일이 존재하지 않습니다. 경로를 확인하세요.")

# 엑셀 파일 읽기
df1 = pd.read_excel(file1_path)
df2 = pd.read_csv(file2_path, encoding='cp949')

# 주소 정규화 함수
def normalize_address(address):
    if pd.isna(address):
        return address
    address = address.strip()
    address = address.replace('강원특별자치도', '강원도')
    address = address.replace('서울특별시', '서울시')
    if '*' in address:
        return None
    return address

# df1의 주소 정규화
df1['full_address'] = df1[['주소시', '주소군구', '주소동']].astype(str).agg(' '.join, axis=1).apply(normalize_address)
df1 = df1.dropna(subset=['full_address'])

# df2의 주소 정규화
df2['소재지전체주소'] = df2['소재지전체주소'].astype(str).apply(normalize_address)
df2['도로명전체주소'] = df2['도로명전체주소'].astype(str).apply(normalize_address)
df2 = df2.dropna(subset=['소재지전체주소', '도로명전체주소'])

# TF-IDF 벡터화
vectorizer = TfidfVectorizer().fit(df1['full_address'])
tfidf_matrix = vectorizer.transform(df1['full_address'])

# 유사한 주소 매핑
def get_best_match(address, tfidf_matrix, vectorizer, choices, threshold=0.7):
    if pd.isna(address):
        return None
    query_vec = vectorizer.transform([address])
    cosine_similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    best_match_index = cosine_similarities.argmax()
    best_match_score = cosine_similarities[best_match_index]
    if best_match_score >= threshold:
        return choices[best_match_index]
    return None

choices = df1['full_address'].tolist()
df2['matched_address_소재지'] = df2['소재지전체주소'].apply(lambda x: get_best_match(x, tfidf_matrix, vectorizer, choices))
df2['matched_address_도로명'] = df2['도로명전체주소'].apply(lambda x: get_best_match(x, tfidf_matrix, vectorizer, choices))

df2['matched_address'] = df2.apply(lambda x: x['matched_address_소재지'] if pd.notna(x['matched_address_소재지']) else x['matched_address_도로명'], axis=1)

# 매핑되지 않은 항목 확인
unmatched = df2[df2['matched_address'].isna()]
print("매핑되지 않은 항목 수:", len(unmatched))

# 평수 계산을 위한 열 존재 확인 및 병합
merge_columns = ['full_address', '관리지사', 'SP담당']
if '총면적' in df1.columns:
    merge_columns.append('총면적')
if '소재지면적' in df1.columns:
    merge_columns.append('소재지면적')

df_merged = df2.merge(df1[merge_columns], left_on='matched_address', right_on='full_address', how='left', suffixes=('', '_df1'))

# 매핑 결과 확인
print("매핑 후 관리지사 및 SP담당이 비어있는 행 수:", df_merged[['관리지사', 'SP담당']].isna().sum())

# 평수 계산 (1평 = 3.305785 m^2)
def calculate_area(row):
    if '소재지면적' in row and pd.notna(row['소재지면적']):
        return round(row['소재지면적'] / 3.305785, 2)
    elif '총면적' in row and pd.notna(row['총면적']):
        return round(row['총면적'] / 3.305785, 2)
    else:
        return None

df_merged['평수'] = df_merged.apply(calculate_area, axis=1)

# 불필요한 열 제외하고 필요한 열만 선택
columns_to_keep = ['관리지사', 'SP담당', '사업장명', '개방서비스명', '업태구분명', '평수', '소재지전체주소', '도로명전체주소', '소재지전화', '폐업일자', '재개업일자', '영업상태명']
df_filtered = df_merged[columns_to_keep]

# 중복된 항목 제거
df_filtered.drop_duplicates(inplace=True)

# 평수 내림차순 정렬
df_filtered_sorted = df_filtered.sort_values(by='평수', ascending=False)

# NaN 값 처리
df_filtered_sorted['관리지사'] = df_filtered_sorted['관리지사'].fillna('Unknown')
df_filtered_sorted['SP담당'] = df_filtered_sorted['SP담당'].fillna('Unknown')

# 네이버 지도 하이퍼링크 생성 함수
def create_naver_map_link(address):
    return f"https://map.naver.com/v5/search/{address}"

# 엑셀 파일에 네이버 지도 링크 추가하는 함수
def add_naver_map_links(ws, address_col_idx):
    ws.insert_cols(1)  # 첫 번째 열에 열 추가
    ws.cell(row=1, column=1, value="네이버 지도 링크")  # 새 열에 헤더 추가

    for row in range(2, ws.max_row + 1):
        address = ws.cell(row=row, column=address_col_idx + 1).value
        if address:
            link = create_naver_map_link(address)
            ws.cell(row=row, column=1).hyperlink = link
            ws.cell(row=row, column=1).value = "네이버 지도 보기"
            ws.cell(row=row, column=1).style = "Hyperlink"

# 엑셀 파일에 데이터를 저장하고 스타일을 설정하는 함수
def save_to_excel(df, file_path):
    with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
        # 전체 시트 저장
        df.to_excel(writer, index=False, sheet_name='전체')

        # 영업/정상 시트 저장
        df_active = df[df['영업상태명'].isin(['영업/정상'])]
        df_active.to_excel(writer, index=False, sheet_name='영업_정상')

        # 폐업 시트 저장
        df_closed = df[df['영업상태명'] == '폐업']
        df_closed.to_excel(writer, index=False, sheet_name='폐업')

        workbook = writer.book
        thin_border = Border(left=Side(style='thin', color='D3D3D3'),
                             right=Side(style='thin', color='D3D3D3'),
                             top=Side(style='thin', color='D3D3D3'),
                             bottom=Side(style='thin', color='D3D3D3'))
        header_fill = PatternFill(start_color="000080", end_color="000080", fill_type="solid")
        header_font = Font(color="FFFFFF", bold=True)

        # 스타일 설정 함수
        def set_style(worksheet):
            for cell in worksheet[1]:
                cell.fill = header_fill
                cell.font = header_font
                cell.border = thin_border
                cell.alignment = Alignment(horizontal='center', vertical='center')
            for row in worksheet.iter_rows(min_row=2):
                for cell in row:
                    cell.border = thin_border
                    cell.alignment = Alignment(horizontal='center', vertical='center')
            for col in worksheet.columns:
                max_length = 0
                column = col[0].column_letter
                for cell in col:
                    try:
                        if len(str(cell.value)) > max_length:
                            max_length = len(cell.value)
                    except:
                        pass
                adjusted_width = (max_length + 2)
                worksheet.column_dimensions[column].width = adjusted_width

        set_style(writer.sheets['전체'])
        set_style(writer.sheets['영업_정상'])
        set_style(writer.sheets['폐업'])

        # 네이버 지도 링크 추가
        # '소재지전체주소'는 H열이므로 컬럼 인덱스는 8-1=7
        add_naver_map_links(writer.sheets['전체'], 7)
        add_naver_map_links(writer.sheets['영업_정상'], 7)
        add_naver_map_links(writer.sheets['폐업'], 7)

# 전체 데이터를 바탕으로 시각화 및 파일 저장
output_dir = '/content/drive/My Drive/회사업무/인허가자료_08월'
for manager in df_filtered_sorted['관리지사'].unique():
    manager_dir = os.path.join(output_dir, manager)
    os.makedirs(manager_dir, exist_ok=True)
    df_manager = df_filtered_sorted[df_filtered_sorted['관리지사'] == manager]

    for sp in df_manager['SP담당'].unique():
        df_sp = df_manager[df_manager['SP담당'] == sp]
        sp_file_path = os.path.join(manager_dir, f'{manager}_{sp}_0809까지.xlsx')
        save_to_excel(df_sp, sp_file_path)

# 전체 데이터를 저장
total_file_path = os.path.join(output_dir, '0801_09까지 전체_최종결과물.xlsx')
save_to_excel(df_filtered_sorted, total_file_path)

# 전체 데이터를 바탕으로 시각화
status_counts = df_filtered_sorted.groupby(['관리지사', '영업상태명']).size().unstack(fill_value=0)
status_counts = status_counts.loc[status_counts.sum(axis=1).sort_values(ascending=False).index]

# 시각화
fig, ax = plt.subplots(figsize=(14, 10))
status_counts.plot(kind='bar', stacked=True, ax=ax, color=['skyblue', 'salmon'])
ax.set_title('관리지사별 영업상태명 집계현황', fontsize=16)
ax.set_xlabel('관리지사', fontsize=12)
ax.set_ylabel('건수', fontsize=12)
ax.legend(title='영업상태명', fontsize=10, title_fontsize='13')
plt.xticks(rotation=90)
plt.tight_layout()

# 그래프를 이미지로 저장
fig_path = os.path.join(output_dir, '전체_영업상태명_집계현황.png')
fig.savefig(fig_path, bbox_inches='tight')

# 그래프를 출력
plt.show()

print(f"전체 매핑된 파일이 생성되었습니다: {total_file_path}")
print(f"전체 영업상태명 집계현황 그래프가 생성되었습니다: {fig_path}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!pip install ydata-profiling

In [ ]:
import pandas as pd
import os
import glob
import zipfile
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Border, Side, Alignment, PatternFill, Font
from google.colab import drive
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.backends.backend_pdf import PdfPages

# 한글 폰트 설치 및 설정
!apt-get install -y fonts-nanum
plt.rcParams['font.family'] = 'NanumGothic'

# 구글 드라이브를 마운트합니다.
drive.mount('/content/drive', force_remount=True)

# 압축 파일 경로 및 압축 해제 경로 설정
zip_file_path = '/content/drive/My Drive/회사업무/인허가자료_08월/LOCALDATA_NOWMON_CSV 3.zip'  # 실제 파일 경로로 변경
extract_folder = '/content/extracted_data'

# 폴더가 없으면 생성
os.makedirs(extract_folder, exist_ok=True)

# 압축 해제
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
except FileNotFoundError:
    print(f"Error: Specified zip file '{zip_file_path}' not found. Please check the path.")
    exit(1)

# 압축 해제된 폴더에서 모든 CSV 파일 목록 가져오기
all_files = glob.glob(os.path.join(extract_folder, "**/*.csv"), recursive=True)

if len(all_files) == 0:
    print("Warning: No CSV files found in the specified directory. Proceeding with an empty DataFrame.")
    dfs = []  # 빈 리스트로 초기화
else:
    # 각 파일을 읽어서 하나의 DataFrame으로 병합
    dfs = []
    for file in all_files:
        try:
            df = pd.read_csv(file, encoding='cp949', on_bad_lines='skip', dtype=str, low_memory=False)
            # 주소라는 키워드를 포함한 열 확인
            address_columns = [col for col in df.columns if '주소' in col]
            if not address_columns:
                print(f"No address column found in file: {file}")
                continue
            # '서울', '경기', '강원'이 포함된 행 필터링
            df_filtered = df[df[address_columns[0]].str.contains('서울|경기|강원', na=False)]
            dfs.append(df_filtered)
            print(f"Successfully read and filtered file: {file}")
        except Exception as e:
            print(f"Error reading file {file}: {e}")

if len(dfs) == 0:
    concatenated_df = pd.DataFrame()  # 이전 단계에서 파일이 없을 경우 빈 DataFrame 생성
else:
    concatenated_df = pd.concat(dfs, ignore_index=True)
    # 중복된 행 제거: 사업장명, 소재지전체주소, 영업상태명이 동일한 경우 하나만 남기고 제거
    concatenated_df.drop_duplicates(subset=['사업장명', '소재지전체주소', '영업상태명'], inplace=True)

    # '인허가일자' 기준으로 내림차순 정렬
    if '인허가일자' in concatenated_df.columns:
        concatenated_df['인허가일자'] = pd.to_datetime(concatenated_df['인허가일자'], format='%Y%m%d', errors='coerce')
        concatenated_df.sort_values(by='인허가일자', ascending=False, inplace=True)

# 필요한 열만 선택하여 필터링
selected_columns = ['소재지전체주소', '도로명전체주소', '도로명우편번호', '사업장명', '개방서비스명', '인허가일자', '인허가취소', '영업상태명', '폐업일자',
                    '휴업시작일', '휴업종료일', '재개업일자', '소재지전화', '최종수정시점', '업태구분명', '좌표정보(X)', '좌표정보(Y)', '총면적', '소재지면적']

# 존재하는 열만 선택
existing_columns = [col for col in selected_columns if col in concatenated_df.columns]
filtered_df = concatenated_df[existing_columns]

# CSV 파일로 저장
output_dir = '/content/drive/My Drive/회사업무/인허가자료_08월'
os.makedirs(output_dir, exist_ok=True)
output_csv_path = os.path.join(output_dir, '1.0801_09까지변동_병합(서울,경기,강원).csv')
filtered_df.to_csv(output_csv_path, index=False, encoding='cp949')

# 엑셀 파일 경로 설정
file1_path = os.path.join('/content/drive/My Drive/회사업무/행안부자료/0801', '1.영업구역별_주소현행화0725.xlsx')
file2_path = output_csv_path  # 이전 단계에서 생성한 파일 사용

# 파일 경로 확인
if not os.path.exists(file1_path):
    raise FileNotFoundError(f"{file1_path} 파일이 존재하지 않습니다. 경로를 확인하세요.")
if not os.path.exists(file2_path):
    raise FileNotFoundError(f"{file2_path} 파일이 존재하지 않습니다. 경로를 확인하세요.")

# 엑셀 파일 읽기
df1 = pd.read_excel(file1_path)
df2 = pd.read_csv(file2_path, encoding='cp949')

# 주소 정규화 함수
def normalize_address(address):
    if pd.isna(address):
        return address
    address = address.strip()
    address = address.replace('강원특별자치도', '강원도')
    address = address.replace('서울특별시', '서울시')
    if '*' in address:
        return None
    return address

# df1의 주소 정규화
df1['full_address'] = df1[['주소시', '주소군구', '주소동']].astype(str).agg(' '.join, axis=1).apply(normalize_address)
df1 = df1.dropna(subset=['full_address'])

# df2의 주소 정규화
df2['소재지전체주소'] = df2['소재지전체주소'].astype(str).apply(normalize_address)
df2['도로명전체주소'] = df2['도로명전체주소'].astype(str).apply(normalize_address)
df2 = df2.dropna(subset=['소재지전체주소', '도로명전체주소'])

# TF-IDF 벡터화
vectorizer = TfidfVectorizer().fit(df1['full_address'])
tfidf_matrix = vectorizer.transform(df1['full_address'])

# 유사한 주소 매핑
def get_best_match(address, tfidf_matrix, vectorizer, choices, threshold=0.7):
    if pd.isna(address):
        return None
    query_vec = vectorizer.transform([address])
    cosine_similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    best_match_index = cosine_similarities.argmax()
    best_match_score = cosine_similarities[best_match_index]
    if best_match_score >= threshold:
        return choices[best_match_index]
    return None

choices = df1['full_address'].tolist()
df2['matched_address_소재지'] = df2['소재지전체주소'].apply(lambda x: get_best_match(x, tfidf_matrix, vectorizer, choices))
df2['matched_address_도로명'] = df2['도로명전체주소'].apply(lambda x: get_best_match(x, tfidf_matrix, vectorizer, choices))

df2['matched_address'] = df2.apply(lambda x: x['matched_address_소재지'] if pd.notna(x['matched_address_소재지']) else x['matched_address_도로명'], axis=1)

# 매핑되지 않은 항목 확인
unmatched = df2[df2['matched_address'].isna()]
print("매핑되지 않은 항목 수:", len(unmatched))

# 평수 계산을 위한 열 존재 확인 및 병합
merge_columns = ['full_address', '관리지사', 'SP담당']
if '총면적' in df1.columns:
    merge_columns.append('총면적')
if '소재지면적' in df1.columns:
    merge_columns.append('소재지면적')

df_merged = df2.merge(df1[merge_columns], left_on='matched_address', right_on='full_address', how='left', suffixes=('', '_df1'))

# 매핑 결과 확인
print("매핑 후 관리지사 및 SP담당이 비어있는 행 수:", df_merged[['관리지사', 'SP담당']].isna().sum())

# 평수 계산 (1평 = 3.305785 m^2)
def calculate_area(row):
    if '소재지면적' in row and pd.notna(row['소재지면적']):
        return round(row['소재지면적'] / 3.305785, 2)
    elif '총면적' in row and pd.notna(row['총면적']):
        return round(row['총면적'] / 3.305785, 2)
# 평수 계산을 이어서 실행
    else:
        return None

df_merged['평수'] = df_merged.apply(calculate_area, axis=1)

# 불필요한 열 제외하고 필요한 열만 선택
columns_to_keep = ['관리지사', 'SP담당', '사업장명', '개방서비스명', '업태구분명', '평수', '소재지전체주소', '도로명전체주소', '소재지전화', '폐업일자', '재개업일자', '영업상태명']
df_filtered = df_merged[columns_to_keep]

# 중복된 항목 제거
df_filtered.drop_duplicates(inplace=True)

# 평수 내림차순 정렬
df_filtered_sorted = df_filtered.sort_values(by='평수', ascending=False)

# NaN 값 처리
df_filtered_sorted['관리지사'] = df_filtered_sorted['관리지사'].fillna('Unknown')
df_filtered_sorted['SP담당'] = df_filtered_sorted['SP담당'].fillna('Unknown')

# 네이버 지도 하이퍼링크 생성 함수
def create_naver_map_link(address):
    return f"https://map.naver.com/v5/search/{address}"

# 엑셀 파일에 네이버 지도 링크 추가하는 함수
def add_naver_map_links(ws, address_col_idx):
    ws.insert_cols(1)  # 첫 번째 열에 열 추가
    ws.cell(row=1, column=1, value="네이버 지도 링크")  # 새 열에 헤더 추가

    for row in range(2, ws.max_row + 1):
        address = ws.cell(row=row, column=address_col_idx + 1).value
        if address:
            link = create_naver_map_link(address)
            ws.cell(row=row, column=1).hyperlink = link
            ws.cell(row=row, column=1).value = "네이버 지도 보기"
            ws.cell(row=row, column=1).style = "Hyperlink"

# 엑셀 파일에 데이터를 저장하고 스타일을 설정하는 함수
def save_to_excel(df, file_path):
    with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
        # 전체 시트 저장
        df.to_excel(writer, index=False, sheet_name='전체')

        # 영업/정상 시트 저장
        df_active = df[df['영업상태명'].isin(['영업/정상'])]
        df_active.to_excel(writer, index=False, sheet_name='영업_정상')

        # 폐업 시트 저장
        df_closed = df[df['영업상태명'] == '폐업']
        df_closed.to_excel(writer, index=False, sheet_name='폐업')

        workbook = writer.book
        thin_border = Border(left=Side(style='thin', color='D3D3D3'),
                             right=Side(style='thin', color='D3D3D3'),
                             top=Side(style='thin', color='D3D3D3'),
                             bottom=Side(style='thin', color='D3D3D3'))
        header_fill = PatternFill(start_color="000080", end_color="000080", fill_type="solid")
        header_font = Font(color="FFFFFF", bold=True)

        # 스타일 설정 함수
        def set_style(worksheet):
            for cell in worksheet[1]:
                cell.fill = header_fill
                cell.font = header_font
                cell.border = thin_border
                cell.alignment = Alignment(horizontal='center', vertical='center')
            for row in worksheet.iter_rows(min_row=2):
                for cell in row:
                    cell.border = thin_border
                    cell.alignment = Alignment(horizontal='center', vertical='center')
            for col in worksheet.columns:
                max_length = 0
                column = col[0].column_letter
                for cell in col:
                    try:
                        if len(str(cell.value)) > max_length:
                            max_length = len(cell.value)
                    except:
                        pass
                adjusted_width = (max_length + 2)
                worksheet.column_dimensions[column].width = adjusted_width

        set_style(writer.sheets['전체'])
        set_style(writer.sheets['영업_정상'])
        set_style(writer.sheets['폐업'])

        # 네이버 지도 링크 추가
        # '소재지전체주소'는 H열이므로 컬럼 인덱스는 8-1=7
        add_naver_map_links(writer.sheets['전체'], 7)
        add_naver_map_links(writer.sheets['영업_정상'], 7)
        add_naver_map_links(writer.sheets['폐업'], 7)

# 전체 데이터를 바탕으로 시각화 및 파일 저장
output_dir = '/content/drive/My Drive/회사업무/인허가자료_08월'
for manager in df_filtered_sorted['관리지사'].unique():
    manager_dir = os.path.join(output_dir, manager)
    os.makedirs(manager_dir, exist_ok=True)
    df_manager = df_filtered_sorted[df_filtered_sorted['관리지사'] == manager]

    for sp in df_manager['SP담당'].unique():
        df_sp = df_manager[df_manager['SP담당'] == sp]
        sp_file_path = os.path.join(manager_dir, f'{manager}_{sp}_0809까지.xlsx')
        save_to_excel(df_sp, sp_file_path)

# 전체 데이터를 저장
total_file_path = os.path.join(output_dir, '0801_09까지 전체_최종결과물.xlsx')
save_to_excel(df_filtered_sorted, total_file_path)

# 전체 데이터를 바탕으로 시각화
status_counts = df_filtered_sorted.groupby(['관리지사', '영업상태명']).size().unstack(fill_value=0)
status_counts = status_counts.loc[status_counts.sum(axis=1).sort_values(ascending=False).index]

# 시각화 및 PDF 저장
pdf_path = os.path.join(output_dir, '전체_영업상태명_집계현황.pdf')
with PdfPages(pdf_path) as pdf:
    # 시각화 1: 관리지사별 영업상태명 집계현황
    fig1, ax1 = plt.subplots(figsize=(14, 10))
    status_counts.plot(kind='bar', stacked=True, ax=ax1, color=['skyblue', 'salmon'])
    ax1.set_title('관리지사별 영업상태명 집계현황', fontsize=16)
    ax1.set_xlabel('관리지사', fontsize=12)
    ax1.set_ylabel('건수', fontsize=12)
    ax1.legend(title='영업상태명', fontsize=10, title_fontsize='13')
    plt.xticks(rotation=90)
    plt.tight_layout()
    pdf.savefig(fig1)
    plt.close(fig1)

    # 시각화 2: SP담당자별 영업상태명 집계현황
    df_filtered_sorted['SP담당자'] = df_filtered_sorted['관리지사'].str.replace('지사', '') + '-' + df_filtered_sorted['SP담당']
    status_counts_sp = df_filtered_sorted.groupby(['SP담당자', '영업상태명']).size().unstack(fill_value=0)
    status_counts_sp = status_counts_sp.loc[status_counts_sp.sum(axis=1).sort_values(ascending=True).index]

    fig2, ax2 = plt.subplots(figsize=(16, 12))
    status_counts_sp.plot(kind='barh', stacked=True, ax=ax2, color=['#4caf50', '#f44336'])
    ax2.set_title('SP담당자별 영업상태명 집계현황', fontsize=18)
    ax2.set_xlabel('건수', fontsize=14)
    ax2.set_ylabel('SP담당자', fontsize=14)
    plt.tight_layout()
    pdf.savefig(fig2)
    plt.close(fig2)

print(f"전체 영업상태명 집계현황 PDF가 생성되었습니다: {pdf_path}")